# Project Part 3

[![Kaggle](https://kaggle.com/static/images/open-in-kaggle.svg)](https://kaggle.com/kernels/welcome?src=https://github.com/CooperDavis00/CS39AA-Project/blob/main/project-part-3-experiment.ipynb)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/sgeinitz/CS39AA-project/blob/main/project_part1.ipynb)

## 1. Introduction/Background

For project part three  I will be using a couple different song datasets that contain the song title, the artist,  the genre(s) of the song, and I will add some different song attributes to each song. I will also be using my own dataset with my personal spotify data of liked songs, this time also using the same song attributes. This personal dataset will have the same parameters along with a parameter that will say 1 or 0 if I like the song or do not.Initially the liked songs dataset will have a like value of 1 and the other dataset i have not seen will all have like values of 0. Instead of changing those values i am going to add another column called "like prediction' that will change based on if the song genre and attributes match my own tase in music. All in all, I am using song data to predict if I would like a song based on my own liked songs

## 2. Exploratory Data Analysis

for part 3 of my project I will be using this dataset: https://www.kaggle.com/datasets/byomokeshsenapati/spotify-song-attributes

I wanted to use a larger and more refined dataset for this but in the end it was going to take too much time to generate genres and song attributes for a dataset with 100,000 entries. If the script i wrote to format my own dataset were to be used on the dataset of over 100,000 entries it would take over 9 hours to run through. This dataset has over 10,000 entries and already has genre and song attributes so it will do. 


In [172]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.impute import SimpleImputer

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.utils import to_categorical



Here is the dataset found from kaggle linked above. Here i read it into a file and format it so the only values displayed are trackName, artistName, and genre. Then I add a like column populated with all zeros since these are the songs i will be predicting and they start with not liked. 

In [173]:
df = pd.read_csv('https://raw.githubusercontent.com/CooperDavis00/CS39AA-Project/main/Spotify_Song_Attributes.csv')
df = df.drop(['msPlayed','time_signature', 'type', 'id', 'uri', 'track_href', 'analysis_url', 'duration_ms'], axis=1)
df['like'] = 0
df['predicted like'] = 0
df = df.dropna(subset=['genre'])

df = df.drop('key', axis=1)
df = df.drop('mode', axis=1)

df.head()

,trackName,artistName,genre,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,like,predicted like
1,"""In The Hall Of The Mountain King"" from Peer G...",London Symphony Orchestra,british orchestra,0.475,0.130,-17.719,0.0510,0.9160,0.956000,0.1010,0.122,112.241,0,0
2,#BrooklynBloodPop!,SyKo,glitchcore,0.691,0.814,-3.788,0.1170,0.0164,0.000000,0.3660,0.509,132.012,0,0
3,$10,Good Morning,experimental pop,0.624,0.596,-9.804,0.0314,0.4750,0.203000,0.1190,0.896,120.969,0,0
4,(I Just) Died In Your Arms,Cutting Crew,album rock,0.625,0.726,-11.402,0.0444,0.0158,0.000169,0.0625,0.507,124.945,0,0
5,(L)only Child,salem ilese,alt z,0.645,0.611,-5.925,0.1370,0.2900,0.000021,0.2370,0.645,157.475,0,0


Here is my dataset generated from my own spotify playlist with the same columns and i added a like column populated with ones. 


In [174]:
df1 = pd.read_csv('https://raw.githubusercontent.com/CooperDavis00/CS39AA-Project/main/liked_w_features.csv')
df1['like'] = 1
df1['predicted like'] = 0
df1 = df1.drop('Track URI', axis = 1)

df1 = df1.drop('key', axis=1)
df1 = df1.drop('mode', axis=1)


df1.head()

,Track Name,Artist Name(s),genre,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,like,predicted like
0,Gamesofluck,Parcels,"aussietronica, indie soul",0.659,0.757,-5.275,0.0380,0.12800,0.48500,0.0859,0.553,104.997,1,0
1,So Much In Love - Armin van Buuren Remix,"D.O.D, Armin van Buuren","electro house, house, pop dance, uk dance, dut...",0.672,0.769,-5.887,0.0788,0.04770,0.00488,0.4390,0.390,132.083,1,0
2,Envious - Rezident Remix,"Aluna, Rezident","dance pop, german house, melodic house",0.659,0.710,-7.476,0.0341,0.14300,0.17300,0.0900,0.281,120.020,1,0
3,Change Your Mind,Interupt,bassline,0.677,0.864,-5.466,0.0689,0.00571,0.00478,0.3560,0.663,128.055,1,0
4,Isolate,D.O.D,"electro house, house, pop dance, uk dance",0.662,0.884,-4.669,0.0500,0.20900,0.16200,0.0980,0.381,126.040,1,0


The column names do not match up so I need to make sure each column matches. 

In [175]:
# Standardizing column names

df1.rename(columns={'Track Name': 'trackName', 'Artist Name(s)': 'artistName', 'Genre': 'genre'}, inplace=True)


print("Columns in df1:", df1.columns)
print("Columns in df:", df.columns)
X_genre = df[['genre']]


Columns in df1: Index(['trackName', 'artistName', 'genre', 'danceability', 'energy',
       'loudness', 'speechiness', 'acousticness', 'instrumentalness',
       'liveness', 'valence', 'tempo', 'like', 'predicted like'],
      dtype='object')
Columns in df: Index(['trackName', 'artistName', 'genre', 'danceability', 'energy',
       'loudness', 'speechiness', 'acousticness', 'instrumentalness',
       'liveness', 'valence', 'tempo', 'like', 'predicted like'],
      dtype='object')


I am counting the different genres and the amount of times each one is displayed. I then display a graph with the top 15 genres and an other category. 

In [181]:
# Concatenate datasets
combined_df = pd.concat([df1, df], ignore_index=True)

combined_df = combined_df.dropna()


# Option 1: Binary Classification for Similar Genres
combined_df['similar_genres'] = combined_df.groupby('trackName')['genre'].transform(lambda x: set(x) == set(combined_df['genre']))

# Create a new DataFrame for genre classification
genre_df = combined_df[['trackName', 'artistName', 'genre', 'similar_genres']]

# Option 2: Regression for Sound Attributes
sound_attributes = ['danceability', 'energy', 'loudness', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo']

# Calculate pairwise Euclidean distances between sound attributes
from sklearn.metrics.pairwise import euclidean_distances
sound_similarity_matrix = euclidean_distances(combined_df[sound_attributes])

# Sum of distances for each row
combined_df['sound_similarity'] = sound_similarity_matrix.sum(axis=0)

#print(combined_df.head())
print(genre_df.head())


                                  trackName               artistName  \
0                               Gamesofluck                  Parcels   
1  So Much In Love - Armin van Buuren Remix  D.O.D, Armin van Buuren   
2                  Envious - Rezident Remix          Aluna, Rezident   
3                          Change Your Mind                 Interupt   
4                                   Isolate                    D.O.D   

                                               genre  similar_genres  
0                          aussietronica, indie soul           False  
1  electro house, house, pop dance, uk dance, dut...           False  
2             dance pop, german house, melodic house           False  
3                                           bassline           False  
4          electro house, house, pop dance, uk dance           False  


In [182]:
# Create a new DataFrame for genre classification
genre_df = combined_df[['trackName', 'artistName', 'genre', 'similar_genres']]

# Split the genres dataset into training and testing sets without stratification
genres_train, genres_val = train_test_split(genre_df, test_size=0.2, random_state=21)

# Ensure the labels are integers (convert boolean to integer)
genres_train['similar_genres'] = genres_train['similar_genres'].astype(int)
genres_val['similar_genres'] = genres_val['similar_genres'].astype(int)

# Features and target for genre prediction
X_genre_train = genres_train[['trackName', 'artistName', 'genre']]
y_genre_train = genres_train['similar_genres']

X_genre_val = genres_val[['trackName', 'artistName', 'genre']]
y_genre_val = genres_val['similar_genres']

# One-hot encode the 'genre' column and convert to integer
X_genre_train_encoded = pd.get_dummies(X_genre_train, columns=['genre'], drop_first=True, dtype=int)
X_genre_val_encoded = pd.get_dummies(X_genre_val, columns=['genre'], drop_first=True, dtype=int)

# Build a model for genre classification
model_genre = Sequential()
model_genre.add(Dense(128, activation='relu', input_dim=X_genre_train_encoded.shape[1]))
model_genre.add(Dropout(0.5))
model_genre.add(Dense(64, activation='relu'))
model_genre.add(Dense(2, activation='softmax'))  # Use 2 output neurons for binary classification
model_genre.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Convert labels to categorical
y_train_genre_encoded = to_categorical(y_genre_train, num_classes=2)
y_val_genre_encoded = to_categorical(y_genre_val, num_classes=2)

# Train the model for genre classification
model_genre.fit(X_genre_train_encoded, y_train_genre_encoded, epochs=10, batch_size=32, validation_data=(X_genre_val_encoded, y_val_genre_encoded))

# Evaluate the model on the validation set for genre classification
accuracy_genre = model_genre.evaluate(X_genre_val_encoded, y_val_genre_encoded)

print("Accuracy for Genre Classification:", accuracy_genre)

NameError: name 'to_categorical' is not defined

In [177]:
# Split the genres dataset into training and testing sets without stratification
genres_train, genres_val = train_test_split(genre_df, test_size=0.2, random_state=21)

# Ensure the labels are integers (convert boolean to integer)
genres_train['similar_genres'] = genres_train['similar_genres'].astype(int)
genres_val['similar_genres'] = genres_val['similar_genres'].astype(int)

# Features and target for genre prediction
X_genre_train = genres_train[['trackName', 'artistName', 'genre']]
y_genre_train = genres_train['similar_genres']

X_genre_val = genres_val[['trackName', 'artistName', 'genre']]
y_genre_val = genres_val['similar_genres']

# One-hot encode the 'genre' column and convert to integer
X_genre_train_encoded = pd.get_dummies(X_genre_train, columns=['genre'], drop_first=True, dtype=int)
X_genre_val_encoded = pd.get_dummies(X_genre_val, columns=['genre'], drop_first=True, dtype=int)


print(X_genre_train_encoded.head())

                     trackName    artistName  genre_N/A, N/A, N/A  \
4805  You Dropped A Bomb On Me  The Gap Band                    0   
1501      Burning in the Skies   Linkin Park                    0   
2581         I Was Never There    The Weeknd                    0   
5586         too many feelings          Ruel                    0   
4553       Valley of the Hills        Sanjoy                    0   

      genre_N/A, N/A, groove room, N/A  genre_N/A, N/A, indie electropop  \
4805                                 0                                 0   
1501                                 0                                 0   
2581                                 0                                 0   
5586                                 0                                 0   
4553                                 0                                 0   

      genre_N/A, N/A, melodic house  genre_N/A, australian indie  \
4805                              0                         

In [178]:
from sklearn.preprocessing import LabelEncoder
# Build a model for genre classification
model_genre = Sequential()
model_genre.add(Dense(128, activation='relu', input_dim=X_genre_train.shape[1]))
model_genre.add(Dropout(0.5))
model_genre.add(Dense(64, activation='relu'))
model_genre.add(Dense(1, activation='sigmoid'))

# Compile the model for genre classification
model_genre.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Use LabelEncoder to convert string labels to numerical values
label_encoder = LabelEncoder()
y_train_genre_encoded = label_encoder.fit_transform(y_genre_train)
y_val_genre_encoded = label_encoder.transform(y_genre_val)

# Train the model for genre classification
model_genre.fit(X_genre_train, y_train_genre_encoded, epochs=10, batch_size=32, validation_data=(X_genre_val, y_val_genre_encoded))

# Evaluate the model on the validation set for genre classification
accuracy_genre = model_genre.evaluate(X_genre_train, y_val_genre_encoded)

print("Accuracy for Genre Classification:", accuracy_genre)

Epoch 1/10


2023-12-05 02:41:14.172440: W tensorflow/core/framework/op_kernel.cc:1805] OP_REQUIRES failed at cast_op.cc:121 : UNIMPLEMENTED: Cast string to float is not supported


UnimplementedError: Graph execution error:

Detected at node 'sequential_29/Cast' defined at (most recent call last):
    File "/Library/Developer/CommandLineTools/Library/Frameworks/Python3.framework/Versions/3.8/lib/python3.8/runpy.py", line 194, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "/Library/Developer/CommandLineTools/Library/Frameworks/Python3.framework/Versions/3.8/lib/python3.8/runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "/Users/cooperdavis/Library/Python/3.8/lib/python/site-packages/ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "/Users/cooperdavis/Library/Python/3.8/lib/python/site-packages/traitlets/config/application.py", line 976, in launch_instance
      app.start()
    File "/Users/cooperdavis/Library/Python/3.8/lib/python/site-packages/ipykernel/kernelapp.py", line 712, in start
      self.io_loop.start()
    File "/Users/cooperdavis/Library/Python/3.8/lib/python

In [ ]:
# Build a model for sound similarity regression
model_sound = Sequential()
model_sound.add(Dense(128, activation='relu', input_dim=X_train_sound.shape[1]))
model_sound.add(Dropout(0.5))
model_sound.add(Dense(64, activation='relu'))
model_sound.add(Dense(1, activation='linear'))

# Compile the model for sound similarity regression
model_sound.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])

# Train the model for sound similarity regression
model_sound.fit(X_train_sound, y_train_sound, epochs=10, batch_size=32, validation_data=(X_val_sound, y_val_sound))

# Evaluate the model on the validation set for sound similarity regression
accuracy_sound = model_sound.evaluate(X_val_sound, y_val_sound)
print("Accuracy for Sound Similarity Regression (MAE):", accuracy_sound)


In [ ]:
# Experiment with hyperparameter tuning, for example, adjusting dropout rate
dropout_rate = 0.2  # Example: Adjust as needed
model_combined_tuned = Sequential()
model_combined_tuned.add(Dense(128, activation='relu', input_dim=X_train.shape[1]))
model_combined_tuned.add(Dropout(dropout_rate))
model_combined_tuned.add(Dense(64, activation='relu'))
model_combined_tuned.add(Dense(1, activation='sigmoid'))  # Output for genre classification
model_combined_tuned.add(Dense(1, activation='linear'))  # Output for sound similarity regression

# Compile the tuned model for combined tasks
model_combined_tuned.compile(optimizer='adam', loss=['binary_crossentropy', 'mean_squared_error'], metrics=['accuracy', 'mae'])

# Train the tuned model for combined tasks
model_combined_tuned.fit(X_train, [y_train_genre, y_train_sound], epochs=10, batch_size=32, validation_data=(X_val, [y_val_genre, y_val_sound]))

# Evaluate the tuned model on the validation set for genre classification and sound similarity regression
accuracy_combined_tuned = model_combined_tuned.evaluate(X_val, [y_val_genre, y_val_sound])
print("Tuned Model Accuracy (Genre Accuracy, Sound MAE):", accuracy_combined_tuned)


In [ ]:

# Build a model for genre classification
model_genre = Sequential()
model_genre.add(Dense(128, activation='relu', input_dim=X_train.shape[1]))
model_genre.add(Dropout(0.5))
model_genre.add(Dense(64, activation='relu'))
model_genre.add(Dense(1, activation='sigmoid'))

# Compile the model for genre classification
model_genre.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model for genre classification
model_genre.fit(X_train, y_train_genre, epochs=10, batch_size=32, validation_data=(X_val, y_val_genre))

# Evaluate the model on the validation set for genre classification
accuracy_genre = model_genre.evaluate(X_val, y_val_genre)

print("Accuracy for Genre Classification:", accuracy_genre)


Epoch 1/10
228/228 [==============================] - 0s 1ms/step - loss: 0.0875 - accuracy: 0.9926 - val_loss: 0.0356 - val_accuracy: 0.9951
Epoch 2/10
228/228 [==============================] - 0s 813us/step - loss: 0.0532 - accuracy: 0.9937 - val_loss: 0.0281 - val_accuracy: 0.9951
Epoch 3/10
228/228 [==============================] - 0s 785us/step - loss: 0.0461 - accuracy: 0.9938 - val_loss: 0.0281 - val_accuracy: 0.9951
Epoch 4/10
228/228 [==============================] - 0s 784us/step - loss: 0.0377 - accuracy: 0.9944 - val_loss: 0.1052 - val_accuracy: 0.9951
Epoch 5/10
228/228 [==============================] - 0s 773us/step - loss: 0.0362 - accuracy: 0.9945 - val_loss: 0.0249 - val_accuracy: 0.9951
Epoch 6/10
228/228 [==============================] - 0s 789us/step - loss: 0.0284 - accuracy: 0.9947 - val_loss: 0.0190 - val_accuracy: 0.9951
Epoch 7/10
228/228 [==============================] - 0s 816us/step - loss: 0.0258 - accuracy: 0.9945 - val_loss: 0.0178 - val_accuracy: 0

In [ ]:
# Handling NaN values in input features
imputer_X = SimpleImputer(strategy='mean')
X_train_imputed = imputer_X.fit_transform(X_train)
X_val_imputed = imputer_X.transform(X_val)

# Ensure the number of features is the same for X_val_imputed and X_val
assert X_val_imputed.shape[1] == X_val.shape[1]

# Handling NaN values in the target variable
imputer_y = SimpleImputer(strategy='mean')
y_train_sound_imputed = imputer_y.fit_transform(y_train_sound.values.reshape(-1, 1)).ravel()
y_val_sound_imputed = imputer_y.transform(y_val_sound.values.reshape(-1, 1)).ravel()

# Build a model for sound similarity regression
model_sound = Sequential()
model_sound.add(Dense(128, activation='relu', input_dim=X_train_imputed.shape[1]))
model_sound.add(Dropout(0.5))
model_sound.add(Dense(64, activation='relu'))
model_sound.add(Dense(1, activation='linear'))

# Compile the model for sound similarity regression
model_sound.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])

# Train the model for sound similarity regression
model_sound.fit(X_train_imputed, y_train_sound_imputed, epochs=10, batch_size=32, validation_data=(X_val_imputed, y_val_sound_imputed))

# Evaluate the model on the validation set for sound similarity regression
accuracy_sound = model_sound.evaluate(X_val_imputed, y_val_sound_imputed)
print("Accuracy for Sound Similarity Regression (MAE):", accuracy_sound)


Epoch 1/10
228/228 [==============================] - 0s 987us/step - loss: nan - mae: nan - val_loss: nan - val_mae: nan
Epoch 2/10
228/228 [==============================] - 0s 1ms/step - loss: nan - mae: nan - val_loss: nan - val_mae: nan
Epoch 3/10
228/228 [==============================] - 0s 761us/step - loss: nan - mae: nan - val_loss: nan - val_mae: nan
Epoch 4/10
228/228 [==============================] - 0s 824us/step - loss: nan - mae: nan - val_loss: nan - val_mae: nan
Epoch 5/10
228/228 [==============================] - 0s 746us/step - loss: nan - mae: nan - val_loss: nan - val_mae: nan
Epoch 6/10
228/228 [==============================] - 0s 736us/step - loss: nan - mae: nan - val_loss: nan - val_mae: nan
Epoch 7/10
228/228 [==============================] - 0s 729us/step - loss: nan - mae: nan - val_loss: nan - val_mae: nan
Epoch 8/10
228/228 [==============================] - 0s 732us/step - loss: nan - mae: nan - val_loss: nan - val_mae: nan
Epoch 9/10
228/228 [======

In [ ]:
model = Sequential()
model.add(Dense(128, activation='relu', input_dim=X_train.shape[1]))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_val, y_val))

# Evaluate the model on the validation set
accuracy = model.evaluate(X_val, y_val)
accuracy

NameError: name 'y_train' is not defined

In this cell i am setting up my variable and training with the logisticRegression model. Then I display the shape and initial accuracy. 


testing the accuracy of the model. 